In [34]:
import pandas as pd
import numpy as np
from Bio import SeqIO
import glob

### metadata: genome

In [3]:
df_0 = pd.read_excel(
    "../dataset/AllSatellitesTables.xlsx", sheet_name="PICI", engine="openpyxl"
)
print(df_0.shape)
print(df_0.columns)

In [ ]:
metadata = pd.read_csv("../dataset/PICI_GenomicNucInfo_060723.csv")
print(metadata.shape)
print(metadata.columns)

In [72]:
metadata["PICI_ID_allsets"] = metadata["PICI_ID"].str.split(".Set", n=1).str[0]
cols = ["PICI_ID_allsets"] + [
    col for col in metadata.columns if col != "PICI_ID_allsets"
]
metadata = metadata[cols]
metadata = metadata.drop(columns=["Unnamed: 0"])

In [ ]:
metadata

In [74]:
duplicated_sets = metadata.groupby("PICI_ID_allsets")["PICI_ID"].nunique()
duplicated_sets = duplicated_sets[duplicated_sets > 1].index
duplicate_rows = metadata[metadata["PICI_ID_allsets"].isin(duplicated_sets)]
duplicate_rows = duplicate_rows.sort_values(["PICI_ID_allsets", "PICI_ID"])

In [79]:
duplicate_rows.to_csv("../dataset/duplicate_picis.csv", index=False)

In [ ]:
print(len(metadata["PICI_ID_allsets"].unique()))
print(len(metadata["PICI_ID"].unique()))
print(len(duplicate_rows["PICI_ID_allsets"].unique()))
print(len(duplicate_rows["PICI_ID"].unique()))
# 1435 - 1286 = 287 - 138 = 149

### metadata: protein

In [ ]:
seqs = list(SeqIO.parse("../dataset/PICI_NucSequences_060723.fst", "fasta"))
print(len(seqs))
print("first sample:")
print(seqs[0])

In [ ]:
seqs_prt_1 = list(
    SeqIO.parse(
        "../dataset/SatelliteProteomes/ACER001.0321.00001.C001.PICI.TypeB.variant0001.Set1.prt",
        "fasta",
    )
)
print(len(seqs_prt_1))
print(seqs_prt_1[0])

In [2]:
data_prt = []

prt_files = glob.glob("../dataset/SatelliteProteomes/*.prt")

for prt_file in prt_files:
    sequences = list(SeqIO.parse(prt_file, "fasta"))

    for seq in sequences:
        # Extract function (part after the last '|')
        function = (
            seq.description.split("|")[-1].strip() if "|" in seq.description else ""
        )

        # Extract PICI_ID (everything before the last underscore in Protein_ID)
        pici_id = "_".join(seq.id.split("_")[:-1])

        # Create dictionary for this entry
        entry = {
            "PICI_ID": pici_id,
            "Protein_ID": seq.id,
            "Function": function,
            "Description": seq.description,
        }

        data_prt.append(entry)

df_prt = pd.DataFrame(data_prt)

In [38]:
df_prt["PICI_ID_allsets"] = df_prt["PICI_ID"].str.split(".Set", n=1).str[0]
cols = ["PICI_ID_allsets"] + [col for col in df_prt.columns if col != "PICI_ID_allsets"]
df_prt = df_prt[cols]

In [40]:
df_prt.to_csv("../dataset/PICI_proteins.csv", index=False)

In [3]:
df_prt = pd.read_csv("../dataset/PICI_proteins.csv")

In [ ]:
df_prt.head()

In [ ]:
print(len(df_prt["PICI_ID_allsets"].unique()))
print(len(df_prt["PICI_ID"].unique()))
print(len(df_prt["Protein_ID"].unique()))
print(len(df_prt["Description"].unique()))
print(len(df_prt["Function"].unique()))

### protein function

In [8]:
function_counts = pd.read_csv("../results/proteins_all_function_counts.tsv", sep="\t")

In [7]:
# function_counts = df_prt["Function"].value_counts().reset_index(name="Count")

In [ ]:
function_counts

In [28]:
integrase_count = function_counts[function_counts["Function"].str.contains("integrase")]
integrase_count.to_csv("../dataset/proteins_integrases.csv", index=False)
alpa_count = function_counts[function_counts["Function"].str.contains("AlpA")]
alpa_count.to_csv("../dataset/proteins_AlpA.csv", index=False)
capsid_count = function_counts[function_counts["Function"].str.contains("capsid")]
capsid_count.to_csv("../dataset/proteins_capsid.csv", index=False)

In [5]:
pici_functions = df_prt.groupby("PICI_ID")["Function"].agg(list)


def check_functions(func_list):
    func_str = " ".join(func_list).lower()
    return "alpa" in func_str


# ("integrase" in func_str) and ('capsid' not in func_str)

filtered_picis = pici_functions[pici_functions.apply(check_functions)]

result_df = pd.DataFrame(
    {"PICI_ID": filtered_picis.index, "All_Functions": filtered_picis.values}
)

In [54]:
result_df.to_csv("../results/picis_integrase_alpa.csv", index=False)

### cfpici

In [ ]:
df_prt = pd.read_csv("../dataset/PICI_proteins.csv")

In [ ]:
# df_prt[
#     df_prt["PICI_ID"].str.contains("CFPICI") & df_prt["Function"].str.contains("capsid")
# ]
len(df_prt[df_prt["PICI_ID"].str.contains("CFPICI")]["PICI_ID"].unique())
# there are 30 cfpici among 1435 picis

In [ ]:
seqs = list(SeqIO.parse("../dataset/PICI_NucSequences_060723.fst", "fasta"))
print(seqs[0])

In [ ]:
for seq in seqs:
    if seq.description == "SIME002.0321.00024.C001.PICI.TypeB.variant0001.SetR1":
        print(seq)
        print(len(seq.seq))
        break

In [ ]:
pici_30 = list(
    [
        "SIME002.0321.00024.C001.PICI.TypeB.variant0001.SetR1",
        "SIME002.0321.00009.C001.PICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00005.C001.PICI.TypeB.variant0001.SetR1",
        "LAPL001.0321.00074.C001.PICI.TypeB.variant0003.SetR1",
        "SIME002.0321.00018.C001.PICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00003.C001.PICI.TypeB.variant0001.SetR1",
        "LAPL001.0321.00073.C001.PICI.TypeB.variant0003.SetR1",
        "SIME002.0321.00002.C001.PICI.TypeB.variant0001.SetR1",
        "STAU002.0321.00280.C001.PICI.TypeB.variant0004.SetR1",
        "SPYA001.0321.00006.C001.PICI.TypeB.variant0001.SetR1",
        "ESCO001.0321.00563.C001.PICI.TypeA.SetR1",
        "ESCO001.0321.00097.C001.PICI.TypeA.SetR1",
        "CIFR005.0321.00052.C001.PICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00004.C001.PICI.TypeB.variant0001.SetR1",
        "CIFR005.0321.00053.C001.PICI.TypeB.variant0001.SetR1",
        "DEVU001.0321.00001.C001.PICI.TypeB.variant0001.SetR1",
        "ESCO001.0321.00309.C001.PICI.TypeB.variant0001.SetR1",
        "ESCO001.0321.01281.C001.PICI.TypeA.SetR1",
        "DEVU001.0321.00001.C001.PICI.TypeB.variant0001.SetR2",
        "PAKO001.0321.00001.C001.PICI.TypeB.variant0004.SetR1",
        "DEVU001.0321.00004.C001.PICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00006.C001.PICI.TypeB.variant0001.SetR1",
        "ESCO001.0321.00448.C001.PICI.TypeA.SetR1",
        "NIHA002.0321.00001.C001.PICI.TypeB.variant0001.SetR1",
        "RHET001.0321.00004.C001.PICI.TypeB.variant0001.SetR1",
        "CIFR005.0321.00101.C001.PICI.TypeB.variant0001.SetR1",
        "STAR001.0321.00001.C001.PICI.TypeB.variant0002.SetR1",
        "PAKO001.0321.00001.C002.PICI.TypeB.variant0004.SetR1",
        "CIAM001.0321.00007.C001.PICI.TypeB.variant0001.SetR1",
        "STSA003.0321.00008.C001.PICI.TypeB.variant0002.SetR1",
    ]
)
cfpici_30 = list(
    [
        "ESCO001.0321.01281.C001.CFPICI.TypeA.SetR1",
        "CIFR005.0321.00053.C001.CFPICI.TypeB.variant0001.SetR1",
        "STSA003.0321.00008.C001.CFPICI.TypeB.variant0002.SetR1",
        "DEVU001.0321.00001.C001.CFPICI.TypeB.variant0001.SetR1",
        "CIFR005.0321.00052.C001.CFPICI.TypeB.variant0001.SetR1",
        "DEVU001.0321.00004.C001.CFPICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00003.C001.CFPICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00004.C001.CFPICI.TypeB.variant0001.SetR1",
        "LAPL001.0321.00074.C001.CFPICI.TypeB.variant0003.SetR1",
        "ESCO001.0321.00097.C001.CFPICI.TypeA.SetR1",
        "DEVU001.0321.00001.C001.CFPICI.TypeB.variant0001.SetR2",
        "SPYA001.0321.00006.C001.CFPICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00005.C001.CFPICI.TypeB.variant0001.SetR1",
        "RHET001.0321.00004.C001.CFPICI.TypeB.variant0001.SetR1",
        "ESCO001.0321.00309.C001.CFPICI.TypeB.variant0001.SetR1",
        "SIME002.0321.00002.C001.CFPICI.TypeB.variant0001.SetR1",
        "PAKO001.0321.00001.C002.CFPICI.TypeB.variant0004.SetR1",
        "STAU002.0321.00280.C001.CFPICI.TypeB.variant0004.SetR1",
        "SIME002.0321.00009.C001.CFPICI.TypeB.variant0001.SetR1",
        "STSI001.0321.00006.C001.CFPICI.TypeB.variant0001.SetR1",
        "CIFR005.0321.00101.C001.CFPICI.TypeB.variant0001.SetR1",
        "LAPL001.0321.00073.C001.CFPICI.TypeB.variant0003.SetR1",
        "SIME002.0321.00024.C001.CFPICI.TypeB.variant0001.SetR1",
        "ESCO001.0321.00563.C001.CFPICI.TypeA.SetR1",
        "STAR001.0321.00001.C001.CFPICI.TypeB.variant0002.SetR1",
        "ESCO001.0321.00448.C001.CFPICI.TypeA.SetR1",
        "CIAM001.0321.00007.C001.CFPICI.TypeB.variant0001.SetR1",
        "PAKO001.0321.00001.C001.CFPICI.TypeB.variant0004.SetR1",
        "NIHA002.0321.00001.C001.CFPICI.TypeB.variant0001.SetR1",
        "SIME002.0321.00018.C001.CFPICI.TypeB.variant0001.SetR1",
    ]
)
print(len(pici_30), len(cfpici_30))

In [ ]:
# keep only 1405 picis from seqs excluding 30 cfpici
seqs = [seq for seq in seqs if seq.id not in cfpici_30]
len(seqs)

In [ ]:
# 1405
# write to fasta
with open("../dataset/PICI_NucSequences_060723_no_cfpici.fst", "w") as f:
    for seq in seqs:
        f.write(f">{seq.id}\n{seq.seq}\n")

### function matrix

In [30]:
df_prt = pd.read_csv("../dataset/PICI_proteins.csv")

In [31]:
function_matrix = pd.pivot_table(
    df_prt,
    values="Protein_ID",
    index="PICI_ID",
    columns="Function",
    aggfunc="count",
    fill_value=0,
)

In [32]:
function_matrix.to_csv("../results/protein_function_matrix_old.csv", index=True)

In [ ]:
print(function_matrix.sum().sum())  # should be same as df_prt.shape[0]
print(function_matrix.max().max())
print(function_matrix.min().min())
print(function_matrix.info())

In [ ]:
# col_mask = function_matrix.columns.str.contains("integrase|alpa|capsid", case=False)
col_mask = function_matrix.columns.str.contains("alpa", case=False)
function_matrix_integrase_alpa_capsid = function_matrix.loc[:, col_mask]

In [ ]:
function_matrix_integrase_alpa_capsid.to_csv(
    "../results/protein_function_matrix_integrase_alpa_capsid.csv", index=True
)

#### new matrix with pharokka annotations

In [ ]:
protein_function_matrix_old = pd.read_csv(
    "../results/protein_function_matrix_old.csv", index_col=0
)
print(protein_function_matrix_old.shape)
print(protein_function_matrix_old.info())
print(protein_function_matrix_old.max().max())
print(protein_function_matrix_old.min().min())

In [ ]:
phanotate_functions = pd.read_csv(
    "../results/pharokka_output/phanotate_functions.tsv", sep="\t"
)
print(phanotate_functions.shape)
print(phanotate_functions.columns)

In [19]:
function_matrix_new_unmerged = pd.pivot_table(
    phanotate_functions,
    values="gene_id",
    index="pici_id",
    columns="function",
    aggfunc="count",
    fill_value=0,
)

In [ ]:
print(function_matrix_new_unmerged.shape)
print(function_matrix_new_unmerged.info())
print(function_matrix_new_unmerged.max().max())
print(function_matrix_new_unmerged.min().min())
function_matrix_new_unmerged.sum().sum()

In [21]:
function_matrix_new_unmerged.to_csv(
    "../results/protein_function_matrix_new_unmerged.csv", index=True
)

In [47]:
function_matrix_new_unmerged = pd.read_csv(
    "../results/protein_function_matrix_new_unmerged.csv", index_col=0
)

In [ ]:
alpa_columns = [
    "AlpA family phage regulatory protein  (translation)",
    "AlpA family transcriptional regulator  (translation)",
]

function_matrix_new = function_matrix_new_unmerged.drop(index=pici_30)
alpa_data = function_matrix[alpa_columns].drop(index=cfpici_30)

print("Shape after removal:")
print("function_matrix_new:", function_matrix_new.shape)
print("alpa_data:", alpa_data.shape)

print("\nIndices identical:", (function_matrix_new.index == alpa_data.index).all())

function_matrix_new = pd.concat([function_matrix_new, alpa_data], axis=1).fillna(0)

print("\nFinal shape:", function_matrix_new.shape)
print("Number of unique indices:", len(function_matrix_new.index.unique()))

In [51]:
function_matrix_new.to_csv(
    "../results/protein_function_matrix_new_merged.csv", index=True
)


In [ ]:
print(function_matrix_new.shape)
print(function_matrix_new.info())
print(function_matrix_new.max().max())
print(function_matrix_new.min().min())
function_matrix_new.sum().sum()

# mash

In [ ]:
from Bio import SeqIO
import os

# Create directory for split files
os.makedirs("../dataset/split_sequences", exist_ok=True)

# Split each sequence into its own file
count = 0
for record in SeqIO.parse("../dataset/PICI_NucSequences_060723.fst", "fasta"):
    count += 1
    # Use the sequence ID as filename, replacing any problematic characters
    safe_filename = record.id.replace("/", "_").replace("\\", "_")
    output_file = f"../dataset/split_sequences/{safe_filename}.fasta"
    with open(output_file, "w") as output_handle:
        SeqIO.write(record, output_handle, "fasta")

print(f"Split {count} sequences into individual files")

In [ ]:
mash_dist = pd.read_csv("../output/mash_distances.tab", sep="\t")
print(mash_dist.shape)
print(mash_dist.columns)

In [24]:
# add column names
mash_dist.columns = ["seq1", "seq2", "distance", "pvalue", "matching_hashes"]

In [ ]:
mash_dist

In [ ]:
import matplotlib.pyplot as plt

plt.hist(mash_dist["distance"], bins=1000)
plt.yscale("log")
plt.show()


# phage_satellites_functions

In [2]:
annotation = pd.read_parquet("../dataset/Phage_and_Satellites_Pann_Pcat_Pcol.pa")

In [ ]:
annotation.head()

In [ ]:
annotation.shape

In [ ]:
print(annotation["acc"].nunique())
print(annotation["name"].nunique())
print(annotation["strand"].nunique())
print(annotation["pann"].nunique())
print(annotation["pcat"].nunique())
print(annotation["pcol"].nunique())
print(annotation["translation"].nunique())
print(annotation["what"].nunique())
print(annotation["translation"].nunique())

In [32]:
# annotation[annotation["pcat"] == "moron, auxiliary metabolic gene and host takeover"]

In [ ]:
print(annotation["pcat"].unique())
print(annotation["pcol"].unique())
print(annotation["what"].unique())

In [ ]:
annotation["pcat"].value_counts()
# leave out: unknown_no_hit
# start with tail

In [6]:
# filter out where pcat is unknown_no_hit
annotation = annotation[annotation["pcat"] != "unknown_no_hit"]

In [ ]:
print(annotation.shape)
print(annotation["translation"].nunique())
print(annotation["pcat"].value_counts())

In [ ]:
duplicated_translations = annotation[annotation.duplicated("translation", keep=False)]
for seq, group in duplicated_translations.groupby("translation"):
    print(f"Protein sequence: {seq[:30]}...")  # Print first 30 chars for brevity
    print(group[["name"]])
    print("-" * 40)

In [35]:
# target_seq = annotation.loc[
#     annotation["name"] == "EZLKZNME_CDS_0010", "translation"
# ].iloc[0]
# same_seq_rows = annotation[annotation["translation"] == target_seq]
# same_seq_rows

In [ ]:
pcat_counts = annotation.groupby("translation")["pcat"].nunique()
multi_pcat_seqs = pcat_counts[pcat_counts > 1].index
rows_with_multi_pcat = annotation[annotation["translation"].isin(multi_pcat_seqs)]
rows_with_multi_pcat
# conclusion: the ones with same translation must have same pcat

"pcat":function, file name
"name": id
"pann": description
"translation": sequence

In [37]:
import os

# Ensure output directory exists (optional)
output_dir = "../dataset/fasta_by_pcat"
os.makedirs(output_dir, exist_ok=True)

for pcat_value in annotation["pcat"].unique():
    # Format the filename: lowercase, replace spaces/commas with underscores, remove other non-alphanumerics
    filename = pcat_value.lower().replace(",", "").replace(" ", "_") + ".fa"
    filepath = os.path.join(output_dir, filename)

    # Select rows for this pcat
    subset = annotation[annotation["pcat"] == pcat_value]

    with open(filepath, "w") as f:
        for _, row in subset.iterrows():
            # Prepare header and sequence
            header = (
                f">{row['name']} {row['pann']}"
                if pd.notna(row["pann"])
                else f">{row['name']}"
            )
            sequence = row["translation"]
            # Write to file
            f.write(f"{header}\n{sequence}\n")

# duplicates

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

seen_sequences = set()
unique_records = []

for record in SeqIO.parse("../dataset/combined_proteins.fa", "fasta"):
    if str(record.seq) not in seen_sequences:
        seen_sequences.add(str(record.seq))
        unique_records.append(record)

SeqIO.write(unique_records, "../dataset/unique_proteins.fa", "fasta")

# ids

In [ ]:
annotation["pcat"].value_counts()

In [10]:
# save the names for each pcat
for pcat, group in annotation.groupby("pcat"):
    group["name"].to_csv(f"../dataset/pcat/{pcat}.csv", index=False)

In [16]:
unique_proteins = list(SeqIO.parse("../dataset/unique_proteins.fa", "fasta"))

In [ ]:
unique_proteins[0]

In [23]:
# save all the ids of unique proteins to csv
list_of_ids = []
for protein in unique_proteins:
    list_of_ids.append(protein.id)
pd.DataFrame(list_of_ids, columns=["id"]).to_csv(
    "../dataset/pcat/all_unique_proteins.csv", index=False
)

# clusters

In [ ]:
import json
import statistics

# Load the cluster mapping
with open("../dataset/protein_cluster_mapping.json", "r") as f:
    protein_to_cluster = json.load(f)

# Count the number of proteins in each cluster
from collections import Counter

sizes = Counter(protein_to_cluster.values()).values()
sizes = list(sizes)

# Print statistics using the statistics module
print(f"Total number of clusters: {len(sizes)}")
print(f"Smallest cluster: {min(sizes)}")
print(f"Largest cluster: {max(sizes)}")
print(f"Average cluster size: {statistics.mean(sizes):.2f}")
print(f"Median cluster size: {statistics.median(sizes)}")
if len(sizes) > 1:
    print(f"Standard deviation: {statistics.stdev(sizes):.2f}")
else:
    print("Standard deviation: N/A (only one cluster)")

In [8]:
import pandas as pd

cluster_sizes = Counter(protein_to_cluster.values())
df = pd.DataFrame(
    {
        "cluster_number": list(cluster_sizes.keys()),
        "cluster_size": list(cluster_sizes.values()),
    }
)
df = df.sort_values("cluster_size", ascending=False)

In [10]:
df.to_csv("../results/cluster_sizes.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt

plt.hist(sizes, bins=1000)
plt.yscale("log")

# bacteria for demonstration 

In [4]:
frik_bacteria_proteins = list(
    SeqIO.parse(
        "../dataset/demonstration_samples/Escherichia_coli_O157_H7_str_FRIK2000/protein.faa",
        "fasta",
    )
)

In [ ]:
print(len(frik_bacteria_proteins))
print(frik_bacteria_proteins[0])

In [6]:
def parse_gff(gff_file):
    records = []
    with open(gff_file) as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            if len(parts) < 9 or parts[2] != "CDS":
                continue
            contig = parts[0]
            start = int(parts[3])
            end = int(parts[4])
            strand = parts[6]
            attributes = parts[8]
            # Try to extract protein_id or ID
            protein_id = None
            for attr in attributes.split(";"):
                if attr.startswith("protein_id="):
                    protein_id = attr.split("=")[1]
                    break
                elif attr.startswith("ID="):
                    protein_id = attr.split("=")[1]
            if protein_id:
                records.append(
                    {
                        "protein_id": protein_id,
                        "contig": contig,
                        "start": start,
                        "end": end,
                        "strand": strand,
                    }
                )
    return pd.DataFrame(records)

In [16]:
gff_df = parse_gff(
    "../dataset/demonstration_samples/Escherichia_coli_O157_H7_str_FRIK2000/genomic.gff"
)
gff_df = gff_df.sort_values(["contig", "start"]).reset_index(drop=True)

In [ ]:
gff_df["contig"].nunique()

In [ ]:
print((gff_df["protein_id"] == sorted(gff_df["protein_id"])).all())
print((gff_df["start"] == sorted(gff_df["start"])).all())
print((gff_df["end"] == sorted(gff_df["end"])).all())
print((gff_df["contig"] == sorted(gff_df["contig"])).all())
print(gff_df.equals(gff_df.sort_values(["contig", "start"])))

In [34]:
gff_df.to_csv(
    "../dataset/demonstration_samples/Escherichia_coli_O157_H7_str_FRIK2000/gff_df.csv",
    index=False,
)

# known segments

## generate samples

In [35]:
annotation = pd.read_parquet("../dataset/Phage_and_Satellites_Pann_Pcat_Pcol.pa")

In [36]:
# see each of what's length
# annotation_pici = annotation[annotation["what"] == "PICI"]
# annotation_cfpici = annotation[annotation["what"] == "CFPICI"]
# annotation_p4 = annotation[annotation["what"] == "P4"]
# annotation_phage = annotation[annotation["what"] == "phage"]

# see the average length of each acc in each what
# annotation_phage["acc"].value_counts().mean()
# print(f"phage: {annotation_phage['acc'].value_counts().describe()}")
# print(f"pici: {annotation_pici['acc'].value_counts().describe()}")
# print(f"cfpici: {annotation_cfpici['acc'].value_counts().describe()}")
# print(f"p4: {annotation_p4['acc'].value_counts().describe()}")

# conclusion: satellites have max length of 44

phage_gene_counts = (
    annotation[annotation["what"] == "phage"]
    .groupby("acc")["name"]
    .nunique()
    .reset_index()
)
phage_gene_counts.columns = ["acc", "gene_num"]

phage_small = phage_gene_counts[
    (phage_gene_counts["gene_num"] >= 10) & (phage_gene_counts["gene_num"] <= 50)
]
phage_small_200 = phage_small.sample(n=200, random_state=42)
annotation_phage_small_200 = annotation[annotation["acc"].isin(phage_small_200["acc"])]
annotation_phage_small_200.to_csv(
    "../dataset/demonstration_samples/known_segments/annotation_phage_small_200.csv",
    index=False,
)

In [ ]:
# annotation = pd.read_parquet("../dataset/Phage_and_Satellites_Pann_Pcat_Pcol.pa")

# # id 200 random samples for each type
# pici_acc_200 = list(
#     np.random.choice(
#         annotation[annotation["what"] == "PICI"]["acc"].unique(), 200, replace=False
#     )
# )
# cf_acc_200 = list(
#     np.random.choice(
#         annotation[annotation["what"] == "CFPICI"]["acc"].unique(), 200, replace=False
#     )
# )
# p4_acc_200 = list(
#     np.random.choice(
#         annotation[annotation["what"] == "P4"]["acc"].unique(), 200, replace=False
#     )
# )
# phage_acc_200 = list(
#     np.random.choice(
#         annotation[annotation["what"] == "phage"]["acc"].unique(), 200, replace=False
#     )
# )
# all_acc_200 = pici_acc_200 + cf_acc_200 + p4_acc_200 + phage_acc_200

# # merge all the samples
# samples = annotation[annotation["acc"].isin(all_acc_200)]
# print(samples["acc"].nunique())

# samples.to_csv(
#     "../dataset/demonstration_samples/known_segments/annotation_200.csv", index=False
# )

## parse seqs

In [37]:
samples = pd.read_csv(
    "../dataset/demonstration_samples/known_segments/annotation_phage_small_200.csv"
)
samples_acc = samples["acc"].unique()
samples_protein_id = samples["name"].unique()

print(len(samples_protein_id))
print(len(samples_acc))
print(samples.shape)

with open(
    "../dataset/demonstration_samples/known_segments/proteins_phage_small_200.faa", "w"
) as f:
    for idx, row in samples.iterrows():
        f.write(f">{row['name']}\n")
        f.write(f"{row['translation']}\n")

6829
200
(6829, 10)
